In [1]:
import pdb
import requests
from bs4 import BeautifulSoup

In [4]:
def find_see_also(soup):
    return soup.find(attrs={"id":"See_also"})


def find_links(soup):
    if soup.name == "ul":
        return soup.find_all("li")
    
    if soup.name == "div" and " ".join(soup["class"]) == "div-col columns column-width":
        return soup.find_all("li")
    
    return find_links(soup.find_next_sibling())


def process_links(links):
    hrefs = []
    for link in links:
        hrefs.append("https://en.wikipedia.org" + link.find('a').get("href"))
    return tuple(hrefs)
    
    
    
def get_article_title(soup):
    return soup.find(attrs={"id":"firstHeading"}).text
    
      
def get_related_links(url):
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content)
    
    title = get_article_title(soup)
    
    see_also = find_see_also(soup)
    
    if see_also:
        links = find_links(see_also.find_parent())
    else: 
        return None
    
    return {
        "title": title,
        "url": url,
        "links": process_links(links)
    }

In [ ]:
def trigger_traversal(entry_url, max_trees=3):
    
    current_tree_count = 0
    seen = {}
    titles = []
    
    max_trees_reached = False
    queue = []

    def traverse_related_links(related_dict, max_trees_reached, current_tree_count, max_trees):
            
        if not related_dict:
            seen[related_dict["url"]] = True

        for url in related_dict['related_links']:
            if not url in seen:
                if (not max_trees_reached) and url:
                    queue.append(url)
        
        #         print(len(queue))
        current_tree_count += 1
        print(current_tree_count)
        if current_tree_count == max_trees:
            max_trees_reached = True
        
        
        while True:
            next_related_dict = get_related_links(queue.pop(0))
            if next_related_dict:
                return traverse_related_links(next_related_dict, max_trees_reached, current_tree_count, max_trees)
            
        
        
        
        
    entry_dict = get_related_links(entry_url)
    
    return traverse_related_links(entry_dict, max_trees_reached, current_tree_count, max_trees)

trigger_traversal("https://en.wikipedia.org/wiki/Brain", max_trees=20)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [5]:
get_related_links("https://en.wikipedia.org/wiki/Brain")

{'title': 'Brain',
 'url': 'https://en.wikipedia.org/wiki/Brain',
 'links': ('https://en.wikipedia.org/wiki/Brain%E2%80%93computer_interface',
  'https://en.wikipedia.org/wiki/Central_nervous_system_disease',
  'https://en.wikipedia.org/wiki/List_of_neuroscience_databases',
  'https://en.wikipedia.org/wiki/Neurological_disorder',
  'https://en.wikipedia.org/wiki/Optogenetics',
  'https://en.wikipedia.org/wiki/Outline_of_neuroscience')}

In [57]:
class WikiScrapper:
    
    seen = {}
    titles = []
    
    tree_count = 0
    max_trees_reached = False
    
    queue = []
    
    data = []
    
    def __init__(self, max_trees=3, verbose=False):
        self.max_trees = max_trees
        self.verbose = verbose

        
    def traverse_from(self, related_dict):
        print(related_dict.get("title"))
        if not related_dict:
            return
        
        self.seen[related_dict['title']] = True
        self.titles.append(related_dict['title'])
        
        
        if not self.max_trees_reached:
            for link in related_dict['links']:
                current_page = get_related_links(link)
                if current_page and not self.seen.get(current_page['title']):
                    if self.tree_count < self.max_trees:
                        self.data.append(current_page)
                        self.queue.append(current_page)
                        self.tree_count += 1
                    else:
                        self.max_trees_reached = True
        if self.verbose:
            print("Queue Length:\t\t", len(self.queue))
            print("Trees Traversed:\t", self.tree_count)
            print("max_trees_reached:\t", self.max_trees_reached)
            print("\n\n")
        
        if self.queue:
            self.traverse_from(self.queue.pop(0))
        
    def get(self, item="titles"):
        if item == "titles":
            return (self.titles)
        elif item == "seen":
            return (self.seen)
        elif item == "data":
            return (self.data)

In [58]:
scrapper = WikiScrapper(max_trees=4)

base_page = get_related_links("https://en.wikipedia.org/wiki/Brain")

scrapper.traverse_from(base_page)

len(scrapper.get("data"))

Brain
Brain–computer interface
Central nervous system disease
List of neuroscience databases
Neurological disorder


4